In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys
# sys.path.append('/beegfs/desy/user/akorol/projects/point-cloud-diffusion/')

/tmp/ipykernel_51740/2900625769.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from utils.plotting import *
from utils.plotting import cfg as cfg_plt

import numpy as np
import h5py
import matplotlib.pyplot as plt

100%|██████████| 30/30 [00:00<00:00, 283.54it/s]


## Calculate num points in the point clouds, and occupancy

In [4]:
path = '/beegfs/desy/user/akorol/data/calo-clouds/hdf5/high_granular_grid/validation/10-90GeV_x36_grid_regular.hdf5' # real point clouds
real_showers = h5py.File(path, 'r')['events'][:]

print(len(real_showers))

pos = np.linspace(0, 30, 31)
for i in tqdm(range(30)):
    idx = np.where((real_showers[:, 1] > pos[i]) & (real_showers[:, 1] < pos[i+1]))
    real_showers[:, 1][idx] = layer_bottom_pos[i] + cell_thickness/2

40000


100%|██████████| 30/30 [00:27<00:00,  1.10it/s]


In [8]:
# fake_showers = np.load('/beegfs/desy/user/akorol/projects/point-cloud/DM_new_100s_10-90GeV_with_flow_corrections_quadratic.npy')
# fake_showers = np.load('/beegfs/desy/user/akorol/projects/point-cloud/DM_new_100s_10-90GeV_with_flow_corrections_quadratic_batch.npy') # generated point clouds with CaloClouds

fake_showers = np.load('/beegfs/desy/user/buhmae/6_PointCloudDiffusion/output/kCaloClouds_2023_05_24__14_54_09_heun18_10-90GeV_40k.npy')
fake_showers[:, -1] = fake_showers[:, -1] / 1000
print(fake_showers.shape)


(40000, 4, 6000)


In [9]:
events = get_projections(real_showers, MAP, layer_bottom_pos)       # get projection of a point clouds to the detector grid
events_fake = get_projections(fake_showers, MAP, layer_bottom_pos)  # get projection of a point clouds to the detector grid

100%|██████████| 40000/40000 [02:03<00:00, 324.63it/s]


In [10]:
_, occ_real, _, _, _ = get_features(events, thr=0.1)      # get event level features
_, occ_fake, _, _, _ = get_features(events_fake, thr=0.1) # get event level features

100%|██████████| 40000/40000 [02:06<00:00, 316.87it/s]


In [11]:
num_hits_real = (real_showers[:, -1, :] > 0).sum(axis=1)  # get num points in the point clouds
num_hits_fakel = (fake_showers[:, -1, :] > 0).sum(axis=1) # get num points in the point clouds

In [13]:
# np.save('occ_real.npy', occ_real)
# np.save('num_hits_real.npy', num_hits_real)

# np.save('occ_fake.npy', occ_fake)
# np.save('num_hits_fakel.npy', num_hits_fakel)



In [50]:
# occ_real = np.load('occ_real.npy')
# num_hits_real = np.load('num_hits_real.npy')

# occ_fake = np.load('occ_fake.npy')
# num_hits_fakel = np.load('num_hits_fakel.npy')

In [12]:
coef = np.polyfit(num_hits_real, occ_real, 3)
poly1d_fn = np.poly1d(coef)

coef_2 = np.polyfit(num_hits_fakel, occ_fake, 3)
poly1d_fn_2 = np.poly1d(coef_2) 

In [13]:
plt.figure(figsize=(10, 10))
plt.scatter(num_hits_real, occ_real, s=0.1, alpha=0.5, label='GEANT4', color='grey')
plt.plot(np.arange(num_hits_real.min(), num_hits_real.max()), poly1d_fn(np.arange(num_hits_real.min(), num_hits_real.max())), '--k', lw=2)
plt.scatter(num_hits_fakel, occ_fake, s=0.1, alpha=0.5, label='DM', color='tab:orange')
plt.plot(np.arange(num_hits_fakel.min(), num_hits_fakel.max()), poly1d_fn_2(np.arange(num_hits_fakel.min(), num_hits_fakel.max())), 'k', lw=2)
plt.ylabel('num. hits', fontsize=20, family='serif')
plt.xlabel('num. clusters', fontsize=20, family='serif')
# plt.xlim(150, 1200)
# plt.ylim(400, 3700)
plt.legend(fontsize=15)
plt.show()

OSError: [Errno 12] Cannot allocate memory

<Figure size 1000x1000 with 1 Axes>

In [14]:
poly1d_fn   # coef_real

poly1d([ 2.39735048e-09, -2.69842295e-05,  2.96136986e-01,  4.89770787e+01])

In [15]:
poly1d_fn_2   # coef_fake

poly1d([ 4.45753201e-09, -4.26483492e-05,  4.03632976e-01,  6.31063427e+01])

In [17]:
def get_scale_factor(num_clusters):
    
    coef_real = np.array([ 2.57988645e-09, -2.94056522e-05,  3.42194568e-01,  5.34968378e+01])
    coef_fake = np.array([ 3.85057207e-09, -4.16463897e-05,  4.19800713e-01,  5.82246858e+01])
    
    poly_fn_real = np.poly1d(coef_real)
    poly_fn_fake = np.poly1d(coef_fake) 
    
    scale_factor = poly_fn_real(num_clusters) / poly_fn_fake(num_clusters)

    return scale_factor

In [18]:
get_scale_factor(np.array([200, 400, 5000]))

array([0.85934084, 0.84574968, 0.84626432])